::::
:::{thebe-button}
:::
::::

# Find objects

Find bubble size using two different approaches.


In [ ]:
from collections.abc import Iterable

from boilercv_pipeline.dfs import limit_group_size
from boilercv_pipeline.models.column import Col, convert, rename
from boilercv_pipeline.models.deps import get_slices
from boilercv_pipeline.models.df import GBC, agg
from boilercv_pipeline.models.subcool import const
from boilercv_pipeline.sets import get_contours_df2, load_video
from boilercv_pipeline.stages.find_objects import FindObjects as Params
from boilercv_pipeline.stages.find_tracks import convert_col
from boilercv_pipeline.units import U
from dev.docs.nbs import get_mode, init
from devtools import pprint
from geopandas import GeoDataFrame, points_from_xy
from matplotlib.pyplot import subplots
from more_itertools import one, only
from numpy import pi, sqrt
from pandas import DataFrame, IndexSlice, NamedAgg, concat
from seaborn import scatterplot
from shapely import LinearRing, Polygon

from boilercv.data import FRAME
from boilercv.images import scale_bool

PARAMS = None
MODE = get_mode()

In [ ]:
PREVIEW_FRAME_COUNT = 50
"""Number of preview frames."""

if isinstance(PARAMS, str):
    params = Params.model_validate_json(PARAMS)
else:
    context = init(mode=MODE)
    if MODE == "docs":
        PREVIEW_FRAME_COUNT = 10
        params = Params(
            context=context,
            compare_with_trackpy=True,
            include_patterns=const.nb_include_patterns,
            slicer_patterns=const.nb_slicer_patterns,
        )
    else:
        params = Params(context=context, only_sample=True)
params.set_display_options()
data = params.data
dfs = only(params.dfs)
C = params.cols

slices = get_slices(one(params.filled_slicers))
frames_slice = slices.get(FRAME, slice(None))
contours = get_contours_df2(one(params.contours)).loc[IndexSlice[frames_slice, :], :]
frames = contours.reset_index()[FRAME].unique()

# # ? Produce reduced-size docs data
# from pathlib import Path
# from boilercv_pipeline.sets import save_video
# save_video(
#     load_video(one(params.filled), slices={FRAME: range(500)}),
#     Path("docs/data/filled/2024-07-18T17-44-35.nc"),
# )
# contours.loc[frames, :].to_hdf(
#     "docs/data/contours/2024-07-18T17-44-35.h5",
#     key="contours",
#     complib="zlib",
#     complevel=9,
# )

M_PER_PX = U.convert(3 / 8, "in", "m") / (202 - 96)
U.define(f"px = {M_PER_PX} m")

PALETTE = {C.approach_tp.val: "red", C.approach.val: "blue"}


def preview(
    df: DataFrame, cols: Iterable[Col] | None = None, index: Col | None = None
) -> DataFrame:
    """Preview a dataframe in the notebook."""
    params.preview(
        cols=cols,
        df=df,
        index=index,
        f=lambda df: df.groupby(C.frame(), **GBC).head(3).head(6),
    )
    return df


pprint(params)

## Data

Load a video of filled contours and the contour loci and plot a composite of all frames to analyze.


In [ ]:
data.plots.composite, ax = subplots()
filled_path = one(params.filled)
with load_video(
    filled_path, slices={FRAME: frames[:: (len(frames) // PREVIEW_FRAME_COUNT)]}
) as video:
    filled_preview = scale_bool(video)
    composite_video = scale_bool(video).max(FRAME).values
    height, width = composite_video.shape[:2]
    ax.imshow(
        ~composite_video, alpha=0.6, extent=(0, width * M_PER_PX, height * M_PER_PX, 0)
    )
    preview_frames = filled_preview.coords[FRAME]
ax.set_xlabel(C.x().replace("px", "m"))
ax.set_ylabel(C.y().replace("px", "m"))


def preview_objects(df: DataFrame):
    """Preview objects."""
    return df.set_index(C.frame()).loc[preview_frames, :].reset_index()

## Find size from filled contours using Trackpy

Use Trackpy to find bubble size given the filled contours.


In [ ]:
if params.compare_with_trackpy:
    from trackpy import batch, quiet

    quiet()

    with load_video(filled_path, slices=slices) as video:
        filled = scale_bool(video)
        trackpy_cols = [*C.trackpy, C.x_tp, C.y_tp]
        data.dfs.trackpy = preview(
            cols=trackpy_cols,
            df=batch(
                frames=filled.values, diameter=params.guess_diameter, characterize=True
            )
            .pipe(C.frame.rename)
            .assign(**{
                C.frame(): lambda df: df[C.frame()].replace(
                    dict(enumerate(filled.frame.values))
                )
            })
            .pipe(rename, trackpy_cols)[[c() for c in trackpy_cols]],
        )

## Find size from contours

The prior approach throws out contour data, instead operating on filled contours. Instead, try using shapely to find size directly from contour data.

### Prepare to find objects

Prepare a dataframe with columns in a certain order, assign contour data to it, and demote the hiearchical indices to plain columns. Count the number of points in each contour and each frame, keeping only those which have enough points to describe a linear ring. Construct a GeoPandas geometry column and operate on it with Shapely to construct linear rings, returning centroids and the representative polygonal area. Also report the number of points in the loci of each contour per frame.


In [ ]:
data.dfs.centroids = preview(
    cols=C.centroids,
    df=contours.reset_index()
    .pipe(rename, C.sources)
    .pipe(limit_group_size, [C.frame(), C.contour()], 3)
    .assign(**{C.geometry(): lambda df: points_from_xy(df[C.x()], df[C.y()])})
    .groupby([C.frame(), C.contour()], **GBC)
    .pipe(
        agg,
        {
            C.centroid(): NamedAgg(C.geometry(), lambda df: LinearRing(df).centroid),
            C.area(): NamedAgg(C.geometry(), lambda df: Polygon(df).area),
        },
    ),
)

Split the centroid point objects into separate named columns that conform to the Trackpy convention. Report the centroids in each frame.


In [ ]:
data.dfs.geo = params.preview(
    cols=C.geo,
    df=data.dfs.centroids.assign(**{
        C.diameter(): lambda df: sqrt(4 * df[C.area()] / pi),
        C.radius_of_gyration(): lambda df: df[C.diameter()] / 4,
        C.size(): lambda df: df[C.radius_of_gyration()],
    }),
)

In [ ]:
data.dfs.dst = preview(
    cols=C.dests,
    df=GeoDataFrame(data.dfs.geo).assign(**{
        C.x(): lambda df: df[C.centroid()].x,
        C.y(): lambda df: df[C.centroid()].y,
    })[[c() for c in C.dests]],
)

## Compare approaches

Compare Trackpy objects with contour objects. Here the guess radius for Trackpy object finding and contour perimeter filtering are matched to produce the same number of objects from each algorithm. Trackpy features more intelligent filtering, but takes much longer. Trackpy's approach for finding local maxima in grayscale images is applied even to binarized images, exhaustively searching for high points in the binary image, adding to execution time.

The percent difference between the approaches is relatively low for this subset, suggesting the contour centroid approach is reasonable.

A warm color palette is used to plot Trackpy objects, and a cool color palette is used to plot contour centroids.


In [ ]:
trackpy_preview = (
    data.dfs.trackpy.pipe(preview_objects)[[C.y_tp(), C.x_tp()]].pipe(
        C.approach_tp.assign
    )
    if params.compare_with_trackpy
    else DataFrame()
)
centroids_preview = (
    data.dfs.dst.pipe(preview_objects)[[C.y(), C.x()]]
    .pipe(C.approach.assign)
    .pipe(convert, [convert_col(C.x, "m"), convert_col(C.y, "m")], U)
)
with params.display_options(scale=1.0):
    scatterplot(
        ax=ax,
        alpha=0.5,
        x=convert_col(C.x, "m")(),
        y=convert_col(C.y, "m")(),
        hue=C.approach(),
        palette=PALETTE,
        data=concat([trackpy_preview, centroids_preview]),
        legend=params.compare_with_trackpy,
    )
    if params.compare_with_trackpy:
        params.move_legend(ax, ncol=1)
data.plots.composite